# Fuzzy C-Means FCM

## 导入数据

In [337]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder #将class(三种花的名字)编码为数值

path = '/home/ysq/桌面/Iris/iris.data'

#读取数据
df = pd.read_csv(path,header=None)
df.head(5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 为类别编码

In [338]:
#为类别编码,将花的名字换为数字
encoder = LabelEncoder().fit(df[4])
df[4] = encoder.transform(df[4])
df.head(5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## 将dataframe转为ndarray

In [339]:
data = df.values
print(type(data))
print(data.shape)

<class 'numpy.ndarray'>
(150, 5)


## 计算欧氏距离

In [340]:
def Euclid_distance(vector1,vector2):
    """
        计算向量1和向量2间的欧氏距离
    """
    foo = vector1-vector2
    return np.linalg.norm(foo,ord=2)#返回L2范数

## 定义聚类中心类

### Iris数据有三类，故需要三个聚类中心

In [341]:
class Center:
    """
        聚类中心类，接受三个向量
    """
    def __init__(self,z0=None,z1=None,z2=None):
        """
            z0,1,2 = None可以使得在生成一个Center类时不必传入参数，之后可以进行传参初始化
        """
        self.z0 = z0
        self.z1 = z1
        self.z2 = z2

### 定义一个列表来接收聚类中心类

In [342]:
center_list = []

### 定义一个列表来接收代价函数

In [343]:
cost_list = []

## Observation:
### 几个数除以其加起来的和返回的数加和为1
### 可用此法产生加和为1的隶属度矩阵的行向量

In [344]:
#eg:
x = 1
y = 2
sum = x+y
x/sum,y/sum

(0.3333333333333333, 0.6666666666666666)

### 初始化隶属度矩阵U，使用随机数产生

In [345]:
import random#随机数

def init_U(m,c):
    """
        data中样本个数m :#data.shape[0]
        跟据提供的聚类簇个数c: #cluster
        产生一个隶属度矩阵 m*c维
        该矩阵每个行向量加起来和为1
        
        思路是先构建一个空母列表（当作空矩阵），再循环产生一些子列表（当作行向量），使用
        列表的append函数，之后再使用numpy的向量化函数np.array()处理该母列表，生成一个
        隶属度矩阵
        
        产生子列表使用随机数的方式，产生c个1至10000之间的随机数，使用Observation，产生
        子列表（隶属度矩阵的行向量）
    """
    
    #一个空的母列表
    U = []
    
    #循环
    for i in range(0,m):
        #行数循环，共m行，即样本个数为m
        #定义一个空的子列表
        row_vector = []
        _sum = 0.0#求和
        for j in range(0,c):
            #列循环，共c列，即聚类簇个数为c
            element = random.randint(1,10000)#产生1～10000之间的随机整数
            row_vector.append(element)
            _sum += element
        
        #使用Observation,行向量每个元素除以其和，让其归一化
        for j in range(0,c):
            row_vector[j] = row_vector[j]/_sum
        
        #获取行向量，并添加至母列表
        U.append(row_vector)
    return U

U = init_U(data.shape[0],3)
#向量化
U = np.array(U)
U.shape

(150, 3)

## Fuzzy C Means类

In [346]:
class FuzzyCMeans_for_iris:
    #构造函数初始化
    def __init__(self,data,U,center_list,cost_list,alpha=2):
        """
            参数说明:
            data为传入的数据集
            U为传入的初始化的隶属度矩阵
            center_list为接收聚类中心的列表
            cost_list为接收代价函数的列表
            alpha为柔性参数（惩罚参数），通常取2,这里直接给他赋值2
        """
        self.data = data
        self.U = U
        self.center_list = center_list
        self.cost_list = cost_list
        self.alpha = alpha
    
    def compute_center_vector(self):
        """
            计算聚类中心向量
            聚类簇个数隐含在隶属度矩阵U中，可根据self.U.shape[1]获取，
            样本个数可以通过self.U.shape[0]获取
        """
        #聚类簇个数
        cluster_number = self.U.shape[1]
        #样本个数
        sample_number = self.U.shape[0]
        #定义一个Center类用来接收中心向量
        c = Center()        
        #寄存中心向量的列表
        foo = []
        #使用循环计算cluster_number个中心向量
        for j in range(cluster_number):
            #计算第j个中心向量     
            #分母
            den = 0.0          
            #分子，初始化为1*4的零向量
            num = np.zeros((1,4))
            
            for i in range(sample_number):
                den += self.U[i][j]**self.alpha
                #注意取data的前四个元素,忽略掉标签！
                num += (self.U[i][j]**self.alpha)*self.data[i][0:4]
                
            #计算完分子和分母，现在计算第j个中心向量
            foo.append(num/den)
        
        c.z0 = foo[0]
        c.z1 = foo[1]
        c.z2 = foo[2]
        #更新center_list
        self.center_list.append(c)
        
    def cost_function(self):
        """
            计算代价函数
        """
        J = 0.0#初始化为0.0
         #聚类簇个数
        cluster_number = self.U.shape[1]
        #样本个数
        sample_number = self.U.shape[0]
        
        #寄存器
        foo = []
        
        #外循环
        for j in range(cluster_number):
            Jj = 0.0
            for i in range(sample_number):
                
                var = self.U[i][j]**self.alpha
                
                #计算第i个样本和当前聚类中心列表的最后一个的第j个的距离,受限于类的类型，无法使用下标，手动分类计算
                if i == 0:
                    dij = Euclid_distance(self.data[i][0:4],self.center_list[-1].z0)
                if i==1:
                    dij = Euclid_distance(self.data[i][0:4],self.center_list[-1].z1)
                if i == 2:
                    dij = Euclid_distance(self.data[i][0:4],self.center_list[-1].z2)
                Jj+=var*(dij**2)
            foo.append(Jj)
        for s in range(len(foo)):
            J+=foo[s]
        print('当前代价函数为',J)
        self.cost_list.append(J)
    
    def stop_compute(self):
        """
            当本次计算的代价函数和上次的代价函数相差不多时停止计算
            定义差量为0.000001
        """
        #差量
        error = 0.0000001
        
        #若当前代价函数列表为空或者只有一个元素，返回False
        if len(self.cost_list)==0 or len(self.cost_list)==1:
            return False
        if self.cost_list[-2]-self.cost_list[-1]<error:
            return True
        else:
            return False
        
    def compute_U_matrix(self):
        """
            当不满足停机条件时，需要更新隶属度矩阵
            此处代码是为了更新隶属度矩阵，判断是否停机交给stop_compute函数
            
            此处借助初始化U时的一个想法，先构建一个母列表，然后循环构建子列表用于接收行向量的元素
            （因为算法每次只能计算处新的U矩阵中的一个元素）,最后使用numpy将母列表向量化得到更新
            后的U (m*c维度，m为样本个数，c为聚类簇个数)
        """
        #聚类簇个数
        cluster_number = self.U.shape[1]
        #样本个数
        sample_number = self.U.shape[0]
        
        #空的母列表
        U = []
        
        for i in range(sample_number):
            row_vector = []#空的子列表
            for j in range(cluster_number):
                num = 1#分子为1
                den = 0.0#分母初始化为0.0,下面使用循环计算分母den
                
                #计算dij
                if j==0:
                    dij = Euclid_distance(self.data[i][0:4],self.center_list[-1].z0)
                if j==1:
                    dij = Euclid_distance(self.data[i][0:4],self.center_list[-1].z1)
                if j==2:
                    dij = Euclid_distance(self.data[i][0:4],self.center_list[-1].z2)                
                #计算分母den
                for k in range(cluster_number):
                    if k==0:
                        dik = Euclid_distance(self.data[i][0:4],self.center_list[-1].z0)
                    if k==1:
                        dik = Euclid_distance(self.data[i][0:4],self.center_list[-1].z1)
                    if k==2:
                        dik = Euclid_distance(self.data[i][0:4],self.center_list[-1].z2)                
                    den += (dij/dik)**(2/(self.alpha-1))
                uij = num/den
                row_vector.append(uij)
            #print(row_vector)
            U.append(row_vector)
        #向量化
        U = np.array(U)
        #更新U
        self.U = U
    

# 进行训练

In [347]:
x = FuzzyCMeans_for_iris(data=data,U=U,center_list=center_list,cost_list=cost_list)
#先计算中心向量
x.compute_center_vector()
x.cost_function()

#对代价进行限制，不满足条件时重新随机生成U，直到满足条件后，开始进行训练
while x.cost_list[-1]>500:
    x.cost_list.clear()
    x.U = np.array(init_U(data.shape[0],3))
    x.compute_center_vector()
    x.cost_function()


#迭代计数器
counter = 0


for i in range(15):
    if x.stop_compute() == True:
        break
    x.compute_U_matrix()
    #print('--after compute:\n',x.U[0],'\n\n')
    x.compute_center_vector()
    x.cost_function()
    counter+=1

print(counter)
print('最近一次的隶属度矩阵:\n',x.U)

当前代价函数为 563.8778698208009
当前代价函数为 409.34069067576274
当前代价函数为 174.80415726978666
当前代价函数为 40.266140161393935
当前代价函数为 47.46252029775301
3
最近一次的隶属度矩阵:
 [[0.01167522 0.01219913 0.97612564]
 [0.02049725 0.0214325  0.95807025]
 [0.02455531 0.02562425 0.94982044]
 [0.02557916 0.02673638 0.94768445]
 [0.01457337 0.01521745 0.97020918]
 [0.03125165 0.03272095 0.9360274 ]
 [0.02330651 0.02432947 0.95236402]
 [0.00717452 0.00750544 0.98532004]
 [0.04716631 0.04921229 0.9036214 ]
 [0.01590552 0.01663881 0.96745567]
 [0.02175346 0.02273866 0.95550788]
 [0.00994059 0.01040243 0.97965698]
 [0.02533258 0.02646751 0.94819991]
 [0.0586022  0.06094477 0.88045303]
 [0.0693376  0.07210352 0.85855889]
 [0.0892691  0.09285492 0.81787598]
 [0.03803245 0.03965401 0.92231354]
 [0.01034813 0.01081675 0.97883512]
 [0.04486813 0.04698282 0.90814905]
 [0.0186918  0.0195283  0.9617799 ]
 [0.01037277 0.01088466 0.97874257]
 [0.01348208 0.01409872 0.9724192 ]
 [0.0439245  0.04565486 0.91042064]
 [0.00204483 0.00214858 

## 根据隶属度矩阵对数据进行分类

In [348]:
#先定义三个接收分类后数据的列表list0,1,2
list0 = []
list1 = []
list2 = []

def classify(U,data,list0,list1,list2):
    """
        分类函数--跟据隶属度矩阵U对data进行分类
        参数说明：U --隶属度矩阵
                data --待分类的数据
                list0 --接收第0类数据的列表
                list1 --接收第1类数据的列表
                list2 --接收第2类数据的列表    
    """
    list0.clear()
    list1.clear()
    list2.clear()
    for i in range(data.shape[0]):
        row_vector = U[i]
        if row_vector[0] > row_vector[1] and row_vector[0] > row_vector[2]:
            list0.append(data[i])
        if row_vector[1] > row_vector[0] and row_vector[1] > row_vector[2]:
            list1.append(data[i])
        if row_vector[2] > row_vector[0] and row_vector[2] > row_vector[1]:
            list2.append(data[i])

classify(x.U,x.data,list0,list1,list2)

In [349]:
list0

[array([7. , 3.2, 4.7, 1.4, 1. ]),
 array([6.9, 3.1, 4.9, 1.5, 1. ]),
 array([6.5, 2.8, 4.6, 1.5, 1. ]),
 array([6.3, 3.3, 4.7, 1.6, 1. ]),
 array([6.6, 2.9, 4.6, 1.3, 1. ]),
 array([5.9, 3.2, 4.8, 1.8, 1. ]),
 array([6.3, 2.5, 4.9, 1.5, 1. ]),
 array([6.8, 2.8, 4.8, 1.4, 1. ]),
 array([6.7, 3. , 5. , 1.7, 1. ]),
 array([6. , 2.7, 5.1, 1.6, 1. ]),
 array([6.7, 3.1, 4.7, 1.5, 1. ]),
 array([6.3, 3.3, 6. , 2.5, 2. ]),
 array([5.8, 2.7, 5.1, 1.9, 2. ]),
 array([7.1, 3. , 5.9, 2.1, 2. ]),
 array([6.3, 2.9, 5.6, 1.8, 2. ]),
 array([6.5, 3. , 5.8, 2.2, 2. ]),
 array([7.6, 3. , 6.6, 2.1, 2. ]),
 array([7.3, 2.9, 6.3, 1.8, 2. ]),
 array([6.7, 2.5, 5.8, 1.8, 2. ]),
 array([7.2, 3.6, 6.1, 2.5, 2. ]),
 array([6.5, 3.2, 5.1, 2. , 2. ]),
 array([6.4, 2.7, 5.3, 1.9, 2. ]),
 array([6.8, 3. , 5.5, 2.1, 2. ]),
 array([5.7, 2.5, 5. , 2. , 2. ]),
 array([5.8, 2.8, 5.1, 2.4, 2. ]),
 array([6.4, 3.2, 5.3, 2.3, 2. ]),
 array([6.5, 3. , 5.5, 1.8, 2. ]),
 array([7.7, 3.8, 6.7, 2.2, 2. ]),
 array([7.7, 2.6, 6.

In [350]:
list1

[array([6.4, 3.2, 4.5, 1.5, 1. ]),
 array([5.5, 2.3, 4. , 1.3, 1. ]),
 array([5.7, 2.8, 4.5, 1.3, 1. ]),
 array([5.2, 2.7, 3.9, 1.4, 1. ]),
 array([5. , 2. , 3.5, 1. , 1. ]),
 array([5.9, 3. , 4.2, 1.5, 1. ]),
 array([6. , 2.2, 4. , 1. , 1. ]),
 array([6.1, 2.9, 4.7, 1.4, 1. ]),
 array([5.6, 2.9, 3.6, 1.3, 1. ]),
 array([6.7, 3.1, 4.4, 1.4, 1. ]),
 array([5.6, 3. , 4.5, 1.5, 1. ]),
 array([5.8, 2.7, 4.1, 1. , 1. ]),
 array([6.2, 2.2, 4.5, 1.5, 1. ]),
 array([5.6, 2.5, 3.9, 1.1, 1. ]),
 array([6.1, 2.8, 4. , 1.3, 1. ]),
 array([6.1, 2.8, 4.7, 1.2, 1. ]),
 array([6.4, 2.9, 4.3, 1.3, 1. ]),
 array([6.6, 3. , 4.4, 1.4, 1. ]),
 array([6. , 2.9, 4.5, 1.5, 1. ]),
 array([5.7, 2.6, 3.5, 1. , 1. ]),
 array([5.5, 2.4, 3.8, 1.1, 1. ]),
 array([5.5, 2.4, 3.7, 1. , 1. ]),
 array([5.8, 2.7, 3.9, 1.2, 1. ]),
 array([5.4, 3. , 4.5, 1.5, 1. ]),
 array([6. , 3.4, 4.5, 1.6, 1. ]),
 array([6.3, 2.3, 4.4, 1.3, 1. ]),
 array([5.6, 3. , 4.1, 1.3, 1. ]),
 array([5.5, 2.5, 4. , 1.3, 1. ]),
 array([5.5, 2.6, 4.

In [351]:
list2

[array([5.1, 3.5, 1.4, 0.2, 0. ]),
 array([4.9, 3. , 1.4, 0.2, 0. ]),
 array([4.7, 3.2, 1.3, 0.2, 0. ]),
 array([4.6, 3.1, 1.5, 0.2, 0. ]),
 array([5. , 3.6, 1.4, 0.2, 0. ]),
 array([5.4, 3.9, 1.7, 0.4, 0. ]),
 array([4.6, 3.4, 1.4, 0.3, 0. ]),
 array([5. , 3.4, 1.5, 0.2, 0. ]),
 array([4.4, 2.9, 1.4, 0.2, 0. ]),
 array([4.9, 3.1, 1.5, 0.1, 0. ]),
 array([5.4, 3.7, 1.5, 0.2, 0. ]),
 array([4.8, 3.4, 1.6, 0.2, 0. ]),
 array([4.8, 3. , 1.4, 0.1, 0. ]),
 array([4.3, 3. , 1.1, 0.1, 0. ]),
 array([5.8, 4. , 1.2, 0.2, 0. ]),
 array([5.7, 4.4, 1.5, 0.4, 0. ]),
 array([5.4, 3.9, 1.3, 0.4, 0. ]),
 array([5.1, 3.5, 1.4, 0.3, 0. ]),
 array([5.7, 3.8, 1.7, 0.3, 0. ]),
 array([5.1, 3.8, 1.5, 0.3, 0. ]),
 array([5.4, 3.4, 1.7, 0.2, 0. ]),
 array([5.1, 3.7, 1.5, 0.4, 0. ]),
 array([4.6, 3.6, 1. , 0.2, 0. ]),
 array([5.1, 3.3, 1.7, 0.5, 0. ]),
 array([4.8, 3.4, 1.9, 0.2, 0. ]),
 array([5. , 3. , 1.6, 0.2, 0. ]),
 array([5. , 3.4, 1.6, 0.4, 0. ]),
 array([5.2, 3.5, 1.5, 0.2, 0. ]),
 array([5.2, 3.4, 1.

## 现在遇到了与Kmeans中一样的问题：
### 分完类后，并不知道这是第几类
### 需要借助统计的方法确认类别

In [352]:
def statistic(_list):
    """
        统计类别以获得标签
    """
    c0 = 0
    c1 = 0
    c2 = 0
    for sample in _list:
        foo = sample[4:5]
        #print(foo)
        if foo==0:
            c0+=1
        elif foo==1:
            c1+=1
        else:
            c2+=1
    #print(c0,c1,c2)
    #Voting
    if c0 > c1 and c0 > c2: 
        return 0
    if c1 > c0 and c1 > c2: 
        return 1
    if c2 > c1 and c2 > c0: 
        return 2

In [353]:
len(list0),len(list1),len(list2)

(60, 38, 52)

In [354]:
print("中心向量:\n")
print(x.center_list[-1].z0,'\n',x.center_list[-1].z1,'\n',x.center_list[-1].z2)

中心向量:

[[6.32838736 2.89862561 5.00626917 1.72960963]] 
 [[6.23955835 2.87759027 4.85825292 1.64623386]] 
 [[5.01240707 3.39831204 1.50860076 0.26098089]]


### 计算分类正确的个数

In [355]:
def correct_number(_list):
    """
        计算某一类的分类正确的个数
    """
    correct = 0
    label = statistic(_list)
    print("第",label,"类")
    for sample in _list:
        #使用sample[4:5]获取其标签
        if sample[4:5] == label:
            correct+=1
    return correct

In [356]:
correct0 = correct_number(list0)
print("正确个数:",correct0)

correct1 = correct_number(list1)
print("正确个数:",correct1)

correct2 = correct_number(list2)
print("正确个数:",correct2)

第 2 类
正确个数: 49
第 1 类
正确个数: 37
第 0 类
正确个数: 50


### 计算指标

In [357]:
OA = (correct0+correct1+correct2)/x.data.shape[0]
print('Overall Accuracy:',OA*100,'%')

AA_0 = correct0/50
AA_1 = correct1/50
AA_2 = correct2/50
AA = (AA_0+AA_1+AA_2)/3

print('Average Accuracy:',AA*100,'%')

Overall Accuracy: 90.66666666666666 %
Average Accuracy: 90.66666666666666 %
